In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import StratifiedKFold


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

import zipfile
with zipfile.ZipFile('/kaggle/input/platesv2/plates.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('/kaggle/working/')
    
print('After zip extraction:')
print(os.listdir("/kaggle/working/"))

data_root = '/kaggle/working/plates/'
print(os.listdir(data_root))

import shutil 
from tqdm import tqdm

train_dir = 'train'
val_dir = 'val'
overall_dir = '/kaggle/working/overall_dataset'
class_names = ['cleaned', 'dirty']

loop_number = 0



os.makedirs(overall_dir, exist_ok=True)
#for class_name in class_names:
#    os.makedirs(os.path.join(overall_dir, class_name), exist_ok=True)
image_num = 0
    
list_of_dirs = ['/kaggle/working/overall_dataset/cleaned','/kaggle/working/overall_dataset/dirty']
list_of_plates = []
for class_name in class_names:   
    for file in os.listdir(data_root+'train/'+class_name):
        file_name = os.fsdecode(file)
        if (file.endswith(".jpg")):  
            shutil.copy(os.path.join(data_root+'train/'+class_name, file_name), 
                        os.path.join(overall_dir, class_name + ' ' + str(image_num) + '.jpg'))
            list_of_plates.append(class_name + ' ' + str(image_num) + '.jpg')
            image_num = image_num + 1

import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import copy
import random
from PIL import Image


from torchvision import transforms, models
from torchvision.transforms import functional
from torch.utils.data import ConcatDataset
from matplotlib.animation import FuncAnimation
batch_size = 8


#for each_dir in list_of_dirs:

            
list_of_classes = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

df = pd.DataFrame({'Model' : [],
               'freezed' : [],
               'augment' : [],
                'num_of_aug' : [],
               'Lr' : [],
               'Ss' : [],
               'Gm' : [],
               'optim' : [],\
              'num_ep' : [],
                'mean_loss' : [],\
                 'file_name' : [],\
                'loop_num' : [],\
              'submission_score' : []})



def set_random_seed(random_seed):
    torch.manual_seed(random_seed)
    random.seed(random_seed)
    np.random.seed(random_seed)
    
set_random_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

strat_kfold = StratifiedKFold(n_splits=5, shuffle=True)

for fold, (train_ids, test_ids) in enumerate(strat_kfold.split(list_of_plates,list_of_classes)):
    
    fold_dir = '/kaggle/working/'+'fold ' + str(fold)
    os.makedirs(fold_dir, exist_ok=True)
    os.makedirs(fold_dir+'/train', exist_ok=True)
    os.makedirs(fold_dir+'/train/cleaned', exist_ok=True)
    os.makedirs(fold_dir+'/train/dirty', exist_ok=True)
    os.makedirs(fold_dir+'/val', exist_ok=True)
    os.makedirs(fold_dir+'/val/cleaned', exist_ok=True)
    os.makedirs(fold_dir+'/val/dirty', exist_ok=True)
    
    for each_elem in train_ids:
        
        if (each_elem < 20):
            shutil.copy(os.path.join(overall_dir, list_of_plates[each_elem]), 
                            os.path.join(fold_dir+'/train/cleaned', list_of_plates[each_elem]))
        else:          
            shutil.copy(os.path.join(overall_dir, list_of_plates[each_elem]), 
                os.path.join(fold_dir+'/train/dirty', list_of_plates[each_elem]))
                        
    for each_elem in test_ids:
        
        if each_elem < 20:
            shutil.copy(os.path.join(overall_dir, list_of_plates[each_elem]), 
                            os.path.join(fold_dir+'/val/cleaned', list_of_plates[each_elem]))
        else:          
            shutil.copy(os.path.join(overall_dir, list_of_plates[each_elem]), 
                os.path.join(fold_dir+'/val/dirty', list_of_plates[each_elem]))
            
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

['platesv2']
After zip extraction:
['.virtual_documents', 'plates', '__MACOSX']
['test', 'train', '.DS_Store']


'test/unknown'

In [2]:
def prepare_data_set(train_dir,val_dir,batch_size,num_of_augmentaions):
    
    list_of_train_datasets = []        
    
    for i in range(num_of_augmentaions):
          
        train_transforms = create_train_transforms()
        train_dataset = torchvision.datasets.ImageFolder(train_dir,train_transforms)   
        
        list_of_train_datasets.append(train_dataset)
            
    train_dataset = ConcatDataset(list_of_train_datasets)
    
    val_transforms = create_val_transforms()
    val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)
    
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
    val_dataloader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)
    
    return train_dataloader, val_dataloader

class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

def show_transformed_image (transrorm, img):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    pic_tensor = transrorm(img)
    pic_array = pic_tensor.permute(1, 2, 0).numpy()* std + mean
    
    rows = 2
    columns = 2
    fig = plt.figure(figsize=(10, 7)) 
    fig.add_subplot(rows, columns, 1) 

    # showing image 
    plt.imshow(img) 
    plt.axis('off') 
    plt.title("Before") 

    # Adds a subplot at the 2nd position 
    fig.add_subplot(rows, columns, 2) 

    # showing image 
    plt.imshow(pic_array) 
    plt.axis('off') 
    plt.title("After") 
        
def show_input(input_tensor, title=''):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    plt.imshow(image.clip(0, 1))
    plt.title(title)
    plt.show()
    plt.pause(0.001)
    
def train_model(device,fold_num,model, loss,optimizer, 
                scheduler,accuracy_history_train,accuracy_history_val,
                epoch_history,train_dataloader,val_dataloader,best_model, num_epochs):    
    best_val_acc = 0
    best_train_acc = 0
    for epoch in tqdm(range(num_epochs)):
        
        epoch_history.append(epoch)
        #print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                dataloader = val_dataloader
                model.eval()   # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward and backward
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                #print(preds_class)
                #print(labels.data)
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)
            if phase == 'train':
                accuracy_history_train.append(epoch_acc.item())
            else:
                if epoch_acc.item() >= best_val_acc:
                    #print('best model - ' + str(epoch_acc.item()) + ' val accuracy')
                    torch.save(model.state_dict(),'/kaggle/working/best_modelx_fold_'+ str(fold_num) +'.pt')
                    best_val_acc = epoch_acc.item() 
                
                    #print(model.state_dict())
                    #best_model = model.state_dict()
                accuracy_history_val.append(epoch_acc.item())

            #print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

    return model

def perform_training_instance(fold_num,
                              batch_size,
                              train_dir,
                              val_dir,
                              num_epochs,
                              num_of_augmentaions,
                              learning_rate,
                              step_size,
                              gamma,
                              optimizer_name,
                              model_name,
                              model_freeze):
    
    model = create_blank_model(model_name, model_freeze)
    loss = torch.nn.CrossEntropyLoss()    
    optimizer = create_optimizer(model,learning_rate,optimizer_name)
    scheduler = create_scheduler(optimizer,step_size,gamma)
    train_dataloader,val_dataloader = prepare_data_set(train_dir,
                                                       val_dir,batch_size,
                                                       num_of_augmentaions)

    epoch_history = []
    accuracy_history_train = []
    accuracy_history_val = []
    best_model = 0
    train_model(device,
                fold_num,model,
                loss, 
                optimizer, 
                scheduler, 
                accuracy_history_train,
                accuracy_history_val,
                epoch_history,
                train_dataloader,
                val_dataloader,
                best_model,
                num_epochs)

    print(max(accuracy_history_train))
    print(max(accuracy_history_val))
    plt.figure(figsize=(20,12))
    plt.plot(epoch_history ,accuracy_history_train,label = 'train')
    plt.plot(epoch_history ,accuracy_history_val,label = 'val')
    plt.legend()
    plt.show()
    

    
def evaluate_model(model_path, val_dir, model_name,model_freeze):
   
    val_transforms = create_val_transforms()
    val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

    val_dataloader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    model = create_blank_model(model_name,model_freeze)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()


    val_predictions = []
    val_onehot = []
    val_img_paths = []
    for inputs, labels in val_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.set_grad_enabled(False):
            preds = model(inputs)
            val_onehot.append(preds.argmax(dim=1))
            m = torch.nn.Softmax(dim=1)
            #preds = m(preds)

        val_predictions.append(m(preds))

    loss_func = torch.nn.CrossEntropyLoss()
    preds = preds.to(device)
    tens = torch.Tensor([0,0,0,0,1,1,1,1]).to(device).type(torch.LongTensor)
    tens = tens.to(device)
    print(loss_func(preds,tens))

    #print(val_predictions)
    #print(val_onehot)
    
def evaluate_ensemble(model_name,model_freeze):
    
    list_of_models = initiate_models(model_name,model_freeze)

    list_of_valDirs = ['/kaggle/working/fold 0/val',
                      '/kaggle/working/fold 1/val',
                      '/kaggle/working/fold 2/val',
                      '/kaggle/working/fold 3/val',
                      '/kaggle/working/fold 4/val']
    val_onehot = []
    val_predictions = []
    list_of_preds = []
    list_of_loss = []
    for index, each_dir in enumerate(list_of_valDirs):
        
        val_transforms = create_val_transforms()
        val_dataset = torchvision.datasets.ImageFolder(each_dir, val_transforms)

        val_dataloader = torch.utils.data.DataLoader(
            val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
        
        
        
        m = torch.nn.Softmax(dim=1)


        for inputs, labels in val_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.set_grad_enabled(False):
                preds = list_of_models[index](inputs)
                val_onehot.append(preds.argmax(dim=1))

                list_of_preds.append(m(preds)) 

            val_predictions.append(m(preds))

            loss_func = torch.nn.CrossEntropyLoss()
            preds = preds.to(device)
            tens = torch.Tensor([0,0,0,0,1,1,1,1]).to(device).type(torch.LongTensor)
            tens = tens.to(device)
            list_of_loss.append(loss_func(preds,tens))
            #print(loss_func(preds,tens))

    #print(list_of_preds)
    #print(torch.stack(list_of_preds).mean(dim=0))
    #print(val_onehot)
    #print(list_of_loss)
    mean_loss = torch.stack(list_of_loss).mean()
    print(mean_loss)
    
    
    return mean_loss
    
def initiate_models(model_name,model_freeze):
    
    list_of_models = []
    
    model0 = create_blank_model(model_name,model_freeze)
    model0.load_state_dict(torch.load('/kaggle/working/best_modelx_fold_0.pt'))
    model0.to(device)
    model0.eval()

    list_of_models.append(model0)

    model1 = create_blank_model(model_name,model_freeze)
    model1.load_state_dict(torch.load('/kaggle/working/best_modelx_fold_1.pt'))
    model1.to(device)
    model1.eval()

    list_of_models.append(model1)


    model2 = create_blank_model(model_name,model_freeze)
    model2.load_state_dict(torch.load('/kaggle/working/best_modelx_fold_2.pt'))
    model2.to(device)
    model2.eval()

    list_of_models.append(model2)

    model3 = create_blank_model(model_name,model_freeze)
    model3.load_state_dict(torch.load('/kaggle/working/best_modelx_fold_3.pt'))
    model3.to(device)
    model3.eval()

    list_of_models.append(model3)

    model4 = create_blank_model(model_name,model_freeze)
    model4.load_state_dict(torch.load('/kaggle/working/best_modelx_fold_4.pt'))
    model4.to(device)
    model4.eval()
    
    list_of_models.append(model4)
    
    return list_of_models

def model_enseble_predict(inputs,list_of_models):
    
    list_of_preds = []
    
    for each_model in list_of_models:
        inputs.to(device)
        each_model.to(device)
        preds = each_model(inputs)
        m = torch.nn.Softmax(dim=1)
        preds = m(preds)
        list_of_preds.append(preds)
 
    return torch.stack(list_of_preds).mean(dim=0)

def create_blank_model(model_name, model_freeze):
    assert model_name in ['resnet18','resnet34','resnet50']
    assert model_freeze in ['except_last_layer','freeze_all_conv', 'unfreeze_last_layer4', 
                           'unfreeze_last_layer4_layer3','unfreeze_all']
    
    if model_name == 'resnet18':
        
        model = models.resnet18(pretrained=True)
        
    elif model_name == 'resnet34':
        
        model = models.resnet34(pretrained=True)
            
    elif model_name == 'resnet50':
            
        model = models.resnet50(pretrained=True)

        # Disable grad for all conv layers
    if model_freeze == 'except_last_layer':
        for param in model.parameters():
            #if isinstance(param, torch.nn.Conv2d):
                param.requires_grad = False
    elif model_freeze == 'freeze_all_conv':
        for param in model.parameters():
            if isinstance(param, torch.nn.Conv2d):
                param.requires_grad = False
                
    elif model_freeze == 'unfreeze_last_layer4':
        for param in model.parameters():
            param.requires_grad = False
            
        for param in model.layer4[0].parameters():  
            param.requires_grad = True
            
    elif model_freeze == 'unfreeze_last_layer4_layer3':
        for param in model.parameters():
            param.requires_grad = False
            
        for param in model.layer4[0].parameters():  
            param.requires_grad = True      
            
        for param in model.layer3[0].parameters():  
            param.requires_grad = True  
            
    elif model_freeze == 'unfreeze_all':
        for param in model.parameters():
            param.requires_grad = True
    
       

    model.fc = torch.nn.Linear(model.fc.in_features, 2)    
    model = model.to(device)
    return model

def create_optimizer(model,learning_rate,optimizer_name):
    assert optimizer_name in ['adam', 'sgd']
    if optimizer_name == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) 
    elif optimizer_name == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) 
    return optimizer
    
def create_scheduler(optimizer,step_size,gamma):
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    return optimizer


def create_train_transforms():

    train_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.CenterCrop((60, 60)),
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])   
    return train_transforms

def add_results_to_dataframe (df,model_name, 
                             model_freeze,
                             tr_transforms,
                             num_of_augmentaions,
                             learning_rate,
                             step_size,
                             gamma,
                             optimizer_name,
                             num_of_epochs,
                             mean_loss,
                             file_name,
                             loop_number):
    
    

    
    df_new_row = pd.DataFrame({'Model' : [model_name],
               'freezed' : [model_freeze],
               'augment' : [str(tr_transforms)],
                'num_of_aug' : [num_of_augmentaions],
               'Lr' : [learning_rate],
               'Ss' : [step_size],
               'Gm' : [gamma],
               'optim' : [optimizer_name],\
              'num_ep' : [num_of_epochs],
                'mean_loss' : [mean_loss.item()],\
                 'file_name' : [file_name],\
                'loop_num' : [loop_number],\
              'submission_score' : [None]})
    

  
    
    return pd.concat([df, df_new_row], ignore_index = True)
   

def create_data_submission_file(model_name,model_freeze,loop_number):
    list_of_models = initiate_models(model_name,model_freeze)
    
    val_transforms = create_val_transforms()
    test_dataset = ImageFolderWithPaths('/kaggle/working/test', val_transforms)

    test_dataloader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)


    test_predictions = []
    test_img_paths = []
    for inputs, labels, paths in tqdm(test_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.set_grad_enabled(False):
            preds = model_enseble_predict(inputs,list_of_models)
        test_predictions.append(
            torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
        test_img_paths.extend(paths)


    test_predictions = np.concatenate(test_predictions)

    submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})
    submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned')
    submission_df['id'] = submission_df['id'].str.replace('/kaggle/working/test/unknown/', '')
    submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
    submission_df.set_index('id', inplace=True)
    submission_df.head(n=6)

    file_name = "submission_resnet" + str(loop_number) + ".csv"
    submission_df.to_csv(file_name)
    
    return file_name
    
def kfold_training(batch_size,
                   num_of_epochs,
                   num_of_augmentaions,
                   learning_rate,
                   step_size,
                   gamma,
                   optimizer_name,
                   model_name,
                   model_freeze,
                   loop_number,
                   df):
    perform_training_instance(0,batch_size,'/kaggle/working/fold 0/train','/kaggle/working/fold 0/val',\
                               num_of_epochs,
                             num_of_augmentaions,learning_rate,step_size,gamma,
                   optimizer_name,
                   model_name,model_freeze)
    perform_training_instance(1,batch_size,'/kaggle/working/fold 1/train','/kaggle/working/fold 1/val',\
                                num_of_epochs,
                             num_of_augmentaions,learning_rate,step_size,gamma,
                   optimizer_name,
                   model_name,model_freeze)
    perform_training_instance(2,batch_size,'/kaggle/working/fold 2/train','/kaggle/working/fold 2/val',\
                               num_of_epochs,
                             num_of_augmentaions,learning_rate,step_size,gamma,
                   optimizer_name,
                   model_name,model_freeze)
    perform_training_instance(3,batch_size,'/kaggle/working/fold 3/train','/kaggle/working/fold 3/val',\
                                num_of_epochs,
                             num_of_augmentaions,learning_rate,step_size,gamma,
                   optimizer_name,
                   model_name,
                             model_freeze)
    perform_training_instance(4,batch_size,'/kaggle/working/fold 4/train','/kaggle/working/fold 4/val',\
                                num_of_epochs,
                             num_of_augmentaions,learning_rate,step_size,gamma,
                   optimizer_name,
                   model_name,model_freeze)
    
    evaluate_model('/kaggle/working/best_modelx_fold_0.pt', '/kaggle/working/fold 0/val'\
                   ,model_name,model_freeze)
    evaluate_model('/kaggle/working/best_modelx_fold_1.pt', '/kaggle/working/fold 1/val'\
                   ,model_name,model_freeze)
    evaluate_model('/kaggle/working/best_modelx_fold_2.pt', '/kaggle/working/fold 2/val'\
                   ,model_name,model_freeze)
    evaluate_model('/kaggle/working/best_modelx_fold_3.pt', '/kaggle/working/fold 3/val'\
                   ,model_name,model_freeze)
    evaluate_model('/kaggle/working/best_modelx_fold_4.pt', '/kaggle/working/fold 4/val'\
                   ,model_name,model_freeze)
    
    mean_loss = evaluate_ensemble(model_name,model_freeze)
    
   
    file_name = create_data_submission_file(model_name,model_freeze,loop_number)
    
    tr_transforms = create_train_transforms()
    result_df = add_results_to_dataframe(df,model_name, 
                             model_freeze,
                             tr_transforms,
                             num_of_augmentaions,
                             learning_rate,
                             step_size,
                             gamma,
                             optimizer_name,
                             num_of_epochs,
                             mean_loss,
                             file_name,
                             loop_number)
    df = result_df
    loop_number = loop_number+1
    return df, loop_number

In [ ]:
def create_train_transforms():

    train_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.CenterCrop((60, 60)),
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])   
    return train_transforms

def create_val_transforms():
    val_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.CenterCrop((60, 60)),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    return val_transforms


learning_rate =1.0e-3
step_size=7
gamma=0.1
model_name = 'resnet34'
model_freeze = 'freeze_all_conv'
num_of_epochs = 60
num_of_augmentaions = 20
optimizer_name = 'sgd'

df, loop_number = kfold_training(batch_size,
                   num_of_epochs,
                   num_of_augmentaions,
                   learning_rate,
                   step_size,
                   gamma,
                   optimizer_name,
                   model_name,
                   model_freeze,
                   loop_number,
                   df)

In [ ]:
def create_train_transforms():

    train_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.CenterCrop((75, 75)),
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])   
    return train_transforms

def create_val_transforms():
    val_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.CenterCrop((75, 75)),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    return val_transforms


learning_rate =1.0e-3
step_size=7
gamma=0.1
model_name = 'resnet34'
model_freeze = 'freeze_all_conv'
num_of_epochs = 60
num_of_augmentaions = 20
optimizer_name = 'sgd'

df, loop_number = kfold_training(batch_size,
                   num_of_epochs,
                   num_of_augmentaions,
                   learning_rate,
                   step_size,
                   gamma,
                   optimizer_name,
                   model_name,
                   model_freeze,
                   loop_number,
                   df)

In [ ]:
def create_train_transforms():

    train_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.CenterCrop((90, 90)),
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])   
    return train_transforms

def create_val_transforms():
    val_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.CenterCrop((90, 90)),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    return val_transforms


learning_rate =1.0e-3
step_size=7
gamma=0.1
model_name = 'resnet34'
model_freeze = 'freeze_all_conv'
num_of_epochs = 60
num_of_augmentaions = 20
optimizer_name = 'sgd'

df, loop_number = kfold_training(batch_size,
                   num_of_epochs,
                   num_of_augmentaions,
                   learning_rate,
                   step_size,
                   gamma,
                   optimizer_name,
                   model_name,
                   model_freeze,
                   loop_number,
                   df)

In [ ]:
df.to_csv('intermediate_result.csv')